# (N) Import libraries

In [ ]:
# connect Colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# to import custom modules
import sys
sys.path.append("/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/scripts")

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.regularizers import l2

In [ ]:
import AddUtil
import DataLoader

In [ ]:
AddUtil.reset_random_seeds()

# (I) AlexNet

In [ ]:
def alexnet_model(
    X_train, 
    y_train,
    X_val, 
    y_val,
    X_test, 
    y_test,
    img_shape=(128, 128, 1), 
    n_classes=10, 
    l2_reg=0.,
	weights=None, 
    ep=64, 
    bs=64, 
    name="alexnet", 
    l_rate=0.001
    ):

    """
    The baseline model is taken from: https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py
    """

    # Ensure reproducibility
    AddUtil.reset_random_seeds()

	# Initialize model
    alexnet = Sequential()

	# Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
		padding='same', kernel_regularizer=l2(l2_reg)))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 3
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(512, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 4
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))

	# Layer 5
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(3072))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

	# Layer 7
    alexnet.add(Dense(4096))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

	# Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('softmax'))
    
    if weights is not None:
        alexnet.load_weights(weights)
  
    # Compile the model
    opt = keras.optimizers.Adam(learning_rate=l_rate)
    alexnet.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Train the model
    history_model = alexnet.fit(
        x=X_train, 
        y=y_train, 
        epochs=ep,
        batch_size=bs, 
        validation_data=(X_val, y_val),
        verbose=1)
    
    # Save accuracy and loss on the trainining and validation samples to the file
    history_dict_model = history_model.history
    AddUtil.save_metrics(history_dict_model,f"history_dict_{name}")

    # Save trained models (weights)
    alexnet.save(f"/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/pretrained_models/{name}")

    # Print training and validation loss and accuracy on the last epoch
    AddUtil.print_metrics(history_dict_model)

    # Evaluate the model on the test data:
    print(AddUtil.fold_evaluate(alexnet, X_test, y_test))

# (II) Data Manipulation

In [ ]:
# Load features from the file
path = "/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/features/spec_feat_with_stand_128_128_11025_1024_1024.npy"
df = DataLoader.create_df(path)
df.info()

In [ ]:
# Create training, validation and test subsets
X_train, y_train, X_val, y_val, X_test, y_test = DataLoader.train_val_test_split(df)

In [ ]:
# cast size of training and validation sample to shape: (128, 128, 1)
X_train_ext = X_train[..., np.newaxis]
X_val_ext = X_val[..., np.newaxis]
print(X_train_ext.shape, X_val_ext.shape)

In [ ]:
# cast size of test sample to shape: (128, 128, 1)
X_test_ext = []
for item in X_test:
    X_test_ext.append(item[..., np.newaxis])

for item in X_test_ext:
    print(item.shape)

# (III) Train the baseline model

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NAME = "alextnet_baseline"

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

# (IV) Influence of learning rate

l_rate = [0.00001, 0.0001, 0.01]

## (4.1) l_rate = 0.0001

In [ ]:
%reset 

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
NAME = "alextnet_lr_0001"

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## (4.2) l_rate = 0.01

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.01
NAME = "alextnet_lr_01"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## (4.2) l_rate = 0.00001

In [ ]:
EPOCHS = 64
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
NAME = "alextnet_lr_00001"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

# (V) Influence of the batch size

bs = [16, 32, 128]

## (5.1) Batch size = 32

In [ ]:
EPOCHS = 64
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NAME = "alextnet_bs_32"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## (5.2) Batch size = 128

In [ ]:
EPOCHS = 64
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NAME = "alextnet_bs_128"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

## (5.3) Batch size = 16

In [ ]:
EPOCHS = 64
BATCH_SIZE = 16
LEARNING_RATE = 0.001
NAME = "alextnet_bs_16"

In [ ]:
print(EPOCHS, BATCH_SIZE, LEARNING_RATE, NAME)

In [ ]:
alexnet_model(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME,
              l_rate=LEARNING_RATE)

# (V) AlexNet model with Stochastic Gradient Descent

## (5.1) Model

In [ ]:
def alexnet_model_sgd(
    X_train, 
    y_train,
    X_val, 
    y_val,
    X_test, 
    y_test,
    img_shape=(128, 128, 1), 
    n_classes=10, 
    l2_reg=0.,
	weights=None, 
    ep=64, 
    bs=64, 
    name="alexnet_sgd", 
    ):

    """
    The baseline model is taken from: https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py
    """

    # Ensure reproducibility
    AddUtil.reset_random_seeds()

	# Initialize model
    alexnet = Sequential()

	# Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape,
		padding='same', kernel_regularizer=l2(l2_reg)))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 3
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(512, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 4
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))

	# Layer 5
    alexnet.add(ZeroPadding2D((1, 1)))
    alexnet.add(Conv2D(1024, (3, 3), padding='same'))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(MaxPooling2D(pool_size=(2, 2)))

	# Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(3072))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

	# Layer 7
    alexnet.add(Dense(4096))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

	# Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(BatchNormalization())
    alexnet.add(Activation('softmax'))
    
    if weights is not None:
        alexnet.load_weights(weights)
  
    # Compile the model
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=10000,
        decay_rate=0.95)
    opt = keras.optimizers.SGD(learning_rate=lr_schedule)

    alexnet.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Train the model
    history_model = alexnet.fit(
        x=X_train, 
        y=y_train, 
        epochs=ep,
        batch_size=bs, 
        validation_data=(X_val, y_val),
        verbose=1)
    
    # Save accuracy and loss on the trainining and validation samples to the file
    history_dict_model = history_model.history
    AddUtil.save_metrics(history_dict_model,f"history_dict_{name}")

    # Save trained models (weights)
    alexnet.save(f"/content/drive/MyDrive/UNIMI/MachineLearning/UrbanSound/pretrained_models/{name}")

    # Print training and validation loss and accuracy on the last epoch
    AddUtil.print_metrics(history_dict_model)

    # Evaluate the model on the test data:
    print(AddUtil.fold_evaluate(alexnet, X_test, y_test))

## (5.2) Training

In [ ]:
EPOCHS = 64
BATCH_SIZE = 32
NAME = "alexnet_sgd"

In [ ]:
print(EPOCHS, BATCH_SIZE, NAME)

In [ ]:
alexnet_model_sgd(X_train_ext, 
              y_train, 
              X_val_ext, 
              y_val, 
              X_test_ext, 
              y_test,
              ep=EPOCHS,
              bs=BATCH_SIZE,
              name=NAME)